In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

# Regression Analysis: Seasonal Effects with Sklearn Linear Regression
In this notebook, you will build a SKLearn linear regression model to predict tungsten ("Close") returns with *lagged* tungsten returns. 

In [10]:
# Futures contract on the tungsten price:

tungsten = pd.read_csv(
    Path("..tungsten.csv"), index_col="Date",parse_dates=True
)
tungsten.head()

FileNotFoundError: [Errno 2] File b'.../tungsten.csv' does not exist: b'.../tungsten.csv'

In [ ]:
# Trim the dataset to begin on June 26 2017
tungsten = tungsten.loc["2017-06-29":, :]
tungsten.head()



# Data Preparation

### Returns

In [ ]:
# Create a series using "Close" price percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s
tungsten['Return'] = tungsten.Close.pct_change() * 100
tungsten = tungsten.replace(-np.inf, np.nan).dropna()
tungsten.tail()

### Lagged Returns 

In [ ]:
# Create a lagged return using the shift function
tungsten['Lagged_Return'] = tungsten.Return.shift()
tungsten = tungsten.replace(-np.inf, np.nan).dropna()
tungsten.tail()

### Train Test Split

In [ ]:
# Create a train/test split for the data using 2018-2019 for testing and the rest for training
train = tungsten[:'2019']
test = tungsten['2019':]


In [ ]:
# Create four dataframes:
# X_train (training set using just the independent variables), X_test (test set of of just the independent variables)
# Y_train (training set using just the "y" variable, i.e., "Futures Return"), Y_test (test set of just the "y" variable):
X_train = train["Lagged_Return"].to_frame()
X_test = test["Lagged_Return"].to_frame()
y_train = train["Return"]
y_test = test["Return"]

In [ ]:
X_train.head()

# Linear Regression Model

In [ ]:
# Create a Linear Regression model and fit it to the training data
from sklearn.linear_model import LinearRegression

# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions using the Testing Data

Note: We want to evaluate the model using data that it has never seen before, in this case: X_test.

In [ ]:
# Make a prediction of "y" values using just the test dataset
predictions = model.predict(X_test)

In [ ]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
Results = y_test.to_frame()
Results["Predicted Return"] = predictions
#Results.head()

In [ ]:
Results

In [ ]:
# Plot the first 20 predictions vs the true values
Results[:20].plot(title='The first 20 predictions vs The true values', figsize=(12, 8))
Results[:20].plot(subplots=True, figsize=(8, 8), title='The first 20 predictions vs The true values')

# Out-of-Sample Performance

Evaluate the model using "out-of-sample" data (X_test and y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
# Calculate the mean_squared_error (MSE) on actual versus predicted test "y" 
mse = mean_squared_error(
    Results["Return"],
    Results["Predicted Return"]
)

# Using that mean-squared-error, calculate the root-mean-squared error (RMSE):
rmse = np.sqrt(mse)
print(f"Out-of-Sample Root Mean Squared Error (RMSE): {rmse}")

# In-Sample Performance

Evaluate the model using in-sample data (X_train and y_train)

In [ ]:
# Construct a dataframe using just the "y" training data:
in_sample_results = y_train.to_frame()

# Add a column of "in-sample" predictions to that dataframe:  
in_sample_results["In-sample Predictions"] = model.predict(X_train)

# Calculate in-sample mean_squared_error (for comparison to out-of-sample)
in_sample_mse = mean_squared_error(
    in_sample_results["Return"],
    in_sample_results["In-sample Predictions"]
)

# Calculate in-sample root mean_squared_error (for comparison to out-of-sample)
in_sample_rmse = np.sqrt(in_sample_mse)
print(f"In-sample Root Mean Squared Error (RMSE): {in_sample_rmse}")

# Conclusions

YOUR CONCLUSIONS HERE!

### lower RMSE is the the sign of fitness of our model, that is why this model perform better with in-Sample data utilization! 